import files

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.model_selection import cross_val_score
# import nltk
# nltk.download('stopwords', quiet=True)
# from nltk.corpus import stopwords
# from nltk.stem.snowball import SnowballStemmer
# from nltk.tokenize import sent_tokenize
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay

load

In [2]:
df = pd.read_csv('../data/tweets.csv', encoding='unicode_escape')

explore

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


rename

In [4]:
df.rename(columns={'tweet_text': 'text',
                   'emotion_in_tweet_is_directed_at': 'company',
                   'is_there_an_emotion_directed_at_a_brand_or_product': 'sentiment'},
          inplace = True)

In [5]:
df[df.text.isna()]

,text,company,sentiment
6,NaN,NaN,No emotion toward brand or product


can't do anything without the text of the tweet

In [6]:
df.dropna(subset=['text'], inplace=True)

In [7]:
df['text'] = df['text'].str.lower()

In [8]:
df.sentiment.value_counts()

sentiment
No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: count, dtype: int64

edit, simplify

In [9]:
df['sentiment'].replace({'No emotion toward brand or product': 'neutral',
                         'Positive emotion': 'positive',
                         'Negative emotion': 'negative',
                         "I can't tell": 'other'
                        }, inplace=True)

In [10]:
df.sentiment.value_counts()

sentiment
neutral     5388
positive    2978
negative     570
other        156
Name: count, dtype: int64

In [11]:
df['company'].replace(['iPad', 'Apple', 'iPad or iPhone App', 'iPhone', 'Other Apple product or service'], 'apple',
                     inplace=True)
df['company'].replace(['Google', 'Other Google product or service', 'Android App', 'Android'], 'google',
                     inplace=True)
df['company'].fillna('other',
                    inplace=True)

In [12]:
df.company.value_counts()

company
other     5801
apple     2409
google     882
Name: count, dtype: int64

deal with missing company after tokenizing, etc. follow code below

In [13]:
apple_words = ['ipad', 'apple', 'iphone', 'itunes', 'ipad2']
google_words = ['google', 'android', 'blogger']

basic_token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(basic_token_pattern)

def company_fix(text, company):
    if company != 'other':
        return company
    else:
        apple, google = False, False
        text_tokenized = tokenizer.tokenize(text)
        for word in apple_words:
            if word in text_tokenized:
                apple = True
                break
        for word in google_words:
            if word in text_tokenized:
                google = True
                break
        if apple & ~google:
            return 'apple'
        elif google & ~apple:
            return 'google'
        elif apple & google:
            return 'both'
        else:
            return 'neither'

df['company'] = df.apply(lambda x: company_fix(x.text, x.company), axis=1)

In [14]:
df.company.value_counts()

company
apple      5398
google     2800
neither     718
both        176
Name: count, dtype: int64

move on to language processing

In [15]:
df['text_tokenized'] = df['text'].apply(lambda x: tokenizer.tokenize(x))

In [ ]:
X, y = df['text'], df['sentiment']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [ ]:
df_freq_dist = FreqDist(df["text_tokenized"].explode())

In [ ]:
def visualize_top_10(freq_dist, title):
    # Extract data for plotting
    top_10 = list(zip(*freq_dist.most_common(25)))
    tokens = top_10[0]
    counts = top_10[1]
    
    # Set up plot and plot data
    fig, ax = plt.subplots()
    ax.bar(tokens, counts)
    
    # Customize plot appearance
    ax.set_title(title)
    ax.set_ylabel("Count")
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.tick_params(axis="x", rotation=90)
    
visualize_top_10(df_freq_dist, "Top 25 Most Common Words")

In [ ]:
tfidf = TfidfVectorizer(max_features=10)
df_vectorized = tfidf.fit_transform(df['text'])